# Packages

In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from constants import Constant as C
from loaders import load_ratings
from models import ContentBased
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge, Lasso
import re


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# How to generate predictions for the hackathon ?
To submit predictions to the hackathon: 
- be sure to consider your latest version of ContentBased. Use your evaluator notebook to assess the quality of your model
- run the make_hackathon_prediction() function in the present notebook with your best feature_method and regressor_method. This will generate a ratings_prediction.csv file
- download ratings_prediction.csv and upload it on the onedrive of your group

In [6]:
def make_hackathon_prediction(feature_method, regressor_method):
    """Generate a prediction file on the test set"""
    
    # 1) load train data - make sure to redirect the DATA_PATH to'data/hackathon'
    assert str(C.DATA_PATH) == 'data/hackathon'
    sp_ratings, df_ratings = load_ratings(surprise_format=True)
    train_set = sp_ratings.build_full_trainset()
    
    # 2) train your ContentBased model on the train set
    content_knn = ContentBased(feature_method, regressor_method)
    content_knn.fit(train_set)
    
    # 3) make predictions on the test set
    df_test = pd.read_csv('data/hackathon/evidence/ratings_test.csv')[C.USER_ITEM_RATINGS]
    test_records = list(df_test.to_records(index=False))
    predictions = content_knn.test(test_records)
    output_predictions = []
    for uid, iid, _, est, _ in predictions:
        output_predictions.append([uid, iid, est])
    df_predictions = pd.DataFrame(data=output_predictions, columns = df_test.columns)

    # 4) dump predictions
    df_predictions.to_csv(f'ratings_predictions.csv', index=False)

    
make_hackathon_prediction("title_length", "ridge_regression")

AssertionError: 